In [ ]:
import os
import json
import pandas as pd
import pathlib
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

from tensorflow import keras
from tensorflow.keras import layers, models

import random
import numpy as np

from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm

from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc, accuracy_score


In [ ]:
# Set TensorFlow seed
tf.random.set_seed(42)

# Set Numpy seed
np.random.seed(42)

# Set Python seed
random.seed(42)

# Set TensorFlow and Keras seed for reproducibility
tf.random.set_seed(42)

In [ ]:
df = pd.read_csv('/content/drive/My Drive/dataset.csv')
df.columns


# Check if any value is equal to 1
colonne_con_valore_1 = (df == 1).any()

# Selects only columns that have at least one value equal to 1
colonne_selezionate = df.loc[:, colonne_con_valore_1]

# Add the last column of the original DataFrame to the selected columns
colonne_selezionate[df.columns[-1]] = df[df.columns[-1]]

df = colonne_selezionate

print(df.columns)

df = df.iloc[0:6000]

SIZE = 244
X_dataset = []

for i in tqdm(range(df.shape[0])):
    img = image.load_img(df.loc[i, 'ImageID'], target_size=(SIZE, SIZE, 3))
    img = image.img_to_array(img)
    img = img/255.
    X_dataset.append(img)

X = np.array(X_dataset)

y = np.array(df.drop(['ImageID'], axis=1))

# Divide the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Divide the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Print the size of the resulting datasets
print("Dimensioni set di addestramento:", X_train.shape, y_train.shape)
print("Dimensioni set di test:", X_test.shape, y_test.shape)
print("Dimensioni set di validazione:", X_val.shape, y_val.shape)

<ipython-input-6-0e26c418cbf6>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  colonne_selezionate[df.columns[-1]] = df[df.columns[-1]]


Index(['33', '34', '35', '36', '37', '38', '39', '40', '161', '162', '163',
       '164', '165', '167', '168', 'ImageID'],
      dtype='object')


100%|██████████| 3000/3000 [09:03<00:00,  5.52it/s]


Dimensioni set di addestramento: (1920, 244, 244, 3) (1920, 15)
Dimensioni set di test: (600, 244, 244, 3) (600, 15)
Dimensioni set di validazione: (480, 244, 244, 3) (480, 15)


In [ ]:
def lossAndAccuracy():
    #plot the training and validation accuracy and loss at each epoch
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

def precisionAndRecall():
    columnValues = [0, 1, 17, 33, 161, 34, 162, 35, 163, 36, 164, 37, 165, 38, 166,
                    39, 167, 40, 168, 49, 50, 65, 66, 67, 81, 82, 83, 84, 85, 86, 97,
                    98, 99, 100, 113, 255]

    y_pred = model.predict(X_test)

    threshold = 0.5
    y_pred_binary = (y_pred > threshold).astype(int)

    from sklearn.metrics import precision_score, recall_score
    precision = precision_score(y_test, y_pred_binary, average='micro')
    recall = recall_score(y_test, y_pred_binary, average='micro')

def checkLabelsItem(row_index):
    # Seleziona la riga e identifica gli indici dei valori a 1
    indices = df.loc[row_index, df.loc[row_index] == 1].index

    # Stampa gli indici dei valori a 1
    print("Valori a 1 nella riga", 1, ":")
    print(indices)


def show_precision_and_roc(model, test_data, test_labels):
    """
    Shows the accuracy and ROC curve of a convolutional network.

    Parameters:
    model: The already trained convolutional network model.
    test_data: Test data to evaluate the model.
    test_labels: Labels corresponding to the test data.

    """
    # Calcola le predizioni del modello sui dati di test
    predictions = model.predict(test_data)

    # Calcola la precisione
    accuracy = accuracy_score(test_labels.argmax(axis=1), predictions.argmax(axis=1))
    print(f"Precisione: {accuracy:.2f}")

    # Calcola la curva ROC per ogni classe
    num_classes = test_labels.shape[1]
    fpr = {}
    tpr = {}
    roc_auc = {}

    for i in range(num_classes):
        fpr[i], tpr[i], _ = roc_curve(test_labels[:, i], predictions[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Calcola la media delle curve ROC per ottenere la curva ROC media
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(num_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
    mean_tpr /= num_classes

    # Calcola l'AUC della curva ROC media
    mean_auc = auc(all_fpr, mean_tpr)

    # Mostra i risultati
    plt.figure()
    plt.plot(all_fpr, mean_tpr, color='b', label=f'Media ROC (area = {mean_auc:.2f})')
    for i in range(num_classes):
        plt.plot(fpr[i], tpr[i], lw=1, label=f'Classe {i} (area = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Curva ROC')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

def calculate_f1_score(model, test_data, test_labels):
    """
     Calculates the F1-score of a convolutional network for multi-class classification.

    Parameters:
    model: The already trained convolutional network model.
    test_data: Test data to evaluate the model.
    test_labels: Labels corresponding to the test data.

    Returns:
    f1_score: The calculated F1-score.
    """

    predictions = model.predict(test_data)

    f1_score_value = f1_score(test_labels.argmax(axis=1), predictions.argmax(axis=1), average='weighted')

    return f1_score_value

In [ ]:
"""
  Using a pre-trained Convolutional Neural Network (CNN)
  as a feature extractor in a deep learning model.
"""

conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(244,244,3)
)


conv_base.summary()

conv_base.trainable = False

In [ ]:
"""
    Experiment 1: one Dense layer with 256 neurons
"""

inputs = keras.Input(shape=(244,244,3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
outputs = layers.Dense(15, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

epochs = 100
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

model.save('vgg16_256.h5')

In [ ]:
"""
    Experiment 2: one Dense layer with 256 neurons and a dropout
"""

inputs = keras.Input(shape=(244,244,3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(15, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


epochs = 100
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)


model.save('vgg16_256_dropout.h5')

In [ ]:
"""
    Experiment 3: one dense layer with 256 neurons
                  and a dropout and GlobalAveragePooling
"""


inputs = keras.Input(shape=(244,244,3))
x = keras.applications.vgg16.preprocess_input(inputs)
x = conv_base(x)
x = GlobalAveragePooling2D()(x)
outputs = layers.Dense(15, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.summary()

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


epochs = 100
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

model.save('vgg16_256_dropout_glob_avg.h5')

In [ ]:
""" Fine Tuning """
vgg_ft_1_layer = models.load_model('vgg16_256_dropout.h5')

for layer in vgg_ft_1_layer.get_layer('vgg16').layers:
    if layer.name in ('block5_conv3'):
        layer.trainable = True
    else:
        layer.trainable = False

for i, layer in enumerate(vgg_ft_1_layer.get_layer('vgg16').layers):
    print(i, layer.name, layer.trainable)

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

epochs = 25
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

In [ ]:
"""
    Experiment 4: remove two blocks dropout model
"""

vgg16_ft_1_block = models.load_model('vgg16_256_dropout.h5')

# Set the granularity of freezing levels.
for layer in vgg16_ft_1_block.get_layer('vgg16').layers:
    if layer.name in ('block5_conv2', 'block5_conv3', 'block5_conv1'):
        layer.trainable = True
    else:
        layer.trainable = False

model.summary()

# Print the trainable status of the levels
for i, layer in enumerate(vgg16_ft_1_block.get_layer('vgg16').layers):
    print(i, layer.name, layer.trainable)


model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

epochs = 25
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

In [ ]:
"""
    Experiment 4bis: remove two blocks dense model
"""

vgg16_ft_1_block = models.load_model('vgg16_256.h5')


for layer in vgg16_ft_1_block.get_layer('vgg16').layers:
    if layer.name in ('block5_conv2', 'block5_conv3', 'block5_conv1', 'block4_conv3', 'block4_conv2', 'block4_conv1'):
        layer.trainable = True
    else:
        layer.trainable = False

model.summary()


for i, layer in enumerate(vgg16_ft_1_block.get_layer('vgg16').layers):
    print(i, layer.name, layer.trainable)



model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


epochs = 25
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

In [ ]:
""" Unfreeze last block dropout """

vgg16_ft_1_block = models.load_model('vgg16_fine_tuned_last_layer_dropout.h5')

for layer in vgg16_ft_1_block.get_layer('vgg16').layers:
    if layer.name in ('block5_conv2', 'block5_conv3', 'block5_conv1'):
        layer.trainable = True
    else:
        layer.trainable = False

model.summary()

for i, layer in enumerate(vgg16_ft_1_block.get_layer('vgg16').layers):
    print(i, layer.name, layer.trainable)

model.compile(optimizer=Adam(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])


epochs = 25
batch_size = 32
history = model.fit(X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_val, y_val))

lossAndAccuracy()
show_precision_and_roc(model, X_test, y_test)

In [ ]:
""" Show Intermediate Activations of a VGG16 Model in Black and White """

# Load a pre-trained VGG16 model
model = keras.applications.VGG16(weights='imagenet', include_top=False)

# Choose an image from your dataset
img_path = 'test2.jpg'
img = keras.preprocessing.image.load_img(img_path, target_size=(244, 244))
img = keras.preprocessing.image.img_to_array(img)
img = np.expand_dims(img, axis=0)

# Get the intermediate layer outputs
layer_names = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']
intermediate_layer_model = keras.Model(inputs=model.input, outputs=[model.get_layer(name).output for name in layer_names])

# Get the intermediate activations for the chosen image
intermediate_activations = intermediate_layer_model.predict(img)

# Visualize the intermediate activations in black and white
for layer_name, activation in zip(layer_names, intermediate_activations):
    plt.figure()
    plt.imshow(activation[0, :, :, 0], cmap='viridis')  # Display the first channel of the activation in black and white
    plt.title(layer_name)
    plt.colorbar()

plt.show()